# Dataset : 

- List of files to read
- File to save to (can be created or appended)
- Arbitrary label associated with the region, mask or whatsoever

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import sys 

sys.path.append('C:/Users/nilso/Documents/EPFL/MA4/PDS Turberg/Rock_Cuttings_Characterisation/')

from Lamp.AttrDict.AttrDict import *

### Train

In [12]:
ifile = 'dataset_train.yaml'

In [13]:
if os.path.isfile(ifile) and os.path.splitext(ifile)[-1] in [".yaml",".yml"]:
    inputs = AttrDict.from_yaml_path(ifile) # change to argv
else:
    raise AssertionError("Wrong input type")

In [14]:
paths = []
labels = []

for key, items in inputs.Train.items():
    for folder in items.Folder:

        path = glob.glob(inputs.rootData + folder + '*.png')
        label = [items.Label] * len(path)

        paths.append(path)
        labels.append(label)

In [15]:
list_of_paths = [item for elem in paths for item in elem]
list_of_labels = [item for elem in labels for item in elem] 

In [18]:
df_train = pd.DataFrame(np.array([list_of_paths,list_of_labels]).T,columns=['Paths','Label'])

In [19]:
df_train['Paths'] = '../' + df_train['Paths'].str.split('/',expand=True,n=9)[9]

In [20]:
df_train_bbox = df_train.loc[df_train.Paths.str.find('bbox',start=-16) > -1, :].reset_index(drop=True)
df_train_mar = df_train.loc[df_train.Paths.str.find('mar',start=-16) > -1, :].reset_index(drop=True)

In [21]:
df_train_bbox.to_csv('train_bbox.csv')
df_train_mar.to_csv('train_mar.csv')

### Test Laboratory

In [38]:
ifile = 'dataset_test_lab.yaml'

In [39]:
if os.path.isfile(ifile) and os.path.splitext(ifile)[-1] in [".yaml",".yml"]:
    inputs = AttrDict.from_yaml_path(ifile) # change to argv
else:
    raise AssertionError("Wrong input type")

In [45]:
paths = []
labels = []

for key, items in inputs.Train.items():
    for folder in items.Folder:

        path = glob.glob(inputs.rootData + folder + '*.png')
        label = [items.Label] * len(path)

        paths.append(path)
        labels.append(label)

In [46]:
list_of_paths = [item for elem in paths for item in elem]
list_of_labels = [item for elem in labels for item in elem]

In [47]:
df_train = pd.DataFrame(np.array([list_of_paths,list_of_labels]).T,columns=['Paths','Label'])

In [48]:
df_train['Paths'] = '../' + df_train['Paths'].str.split('/',expand=True,n=9)[9]

In [49]:
df_train_bbox = df_train.loc[df_train.Paths.str.find('bbox',start=-16) > -1, :].reset_index(drop=True)
df_train_mar = df_train.loc[df_train.Paths.str.find('mar',start=-16) > -1, :].reset_index(drop=True)

In [50]:
df_train_bbox.to_csv('test_lab_bbox.csv')
df_train_mar.to_csv('test_lab_mar.csv')

### Test Boreholes

In [2]:
ifile = 'dataset_test_borehole.yaml'

In [3]:
if os.path.isfile(ifile) and os.path.splitext(ifile)[-1] in [".yaml",".yml"]:
    inputs = AttrDict.from_yaml_path(ifile) # change to argv
else:
    raise AssertionError("Wrong input type")

In [4]:
paths = []
labels = []

for key, items in inputs.Train.items():
    for folder in items.Folder:

        path = glob.glob(inputs.rootData + folder + '*.png')
        label = [items.Label] * len(path)

        paths.append(path)
        labels.append(label)

In [5]:
list_of_paths = [item for elem in paths for item in elem]
list_of_labels = [item for elem in labels for item in elem]

In [6]:
df_train = pd.DataFrame(np.array([list_of_paths,list_of_labels]).T,columns=['Paths','Label'])

In [7]:
df_train['Paths'] = '../' + df_train['Paths'].str.split('/',expand=True,n=9)[9]

In [8]:
df_train_bbox = df_train.loc[df_train.Paths.str.find('bbox',start=-16) > -1, :].reset_index(drop=True)
df_train_mar = df_train.loc[df_train.Paths.str.find('mar',start=-16) > -1, :].reset_index(drop=True)

In [9]:
df_train_bbox.to_csv('test_borehole_bbox.csv')
df_train_mar.to_csv('test_borehole_mar.csv')